Name: Ruotian Jiang ID:8389636738

# 2. Multi-class Classification Using Support Vector Machines

# a

In [92]:
import numpy as np  
import pandas as pd 
from sklearn.model_selection import train_test_split
  
df = pd.read_csv('Frogs_MFCCs.csv',delimiter=',')  
train, test = train_test_split(df, train_size=0.7)
print test
print test.iloc[:,22:25].values


      MFCCs_ 1  MFCCs_ 2  MFCCs_ 3  MFCCs_ 4  MFCCs_ 5  MFCCs_ 6  MFCCs_ 7  \
6967  1.000000  0.414643  0.601373  0.290996 -0.129241  0.129944  0.225549   
6073  1.000000  0.124066  0.326223  0.322246  0.160067  0.175506  0.098002   
3855  1.000000  0.208916  0.214381  0.528076  0.177962  0.027795 -0.143063   
1067  1.000000  0.358286  0.178984  0.315646  0.058655  0.062652  0.157903   
334   1.000000  0.611662  0.623435  0.435339 -0.149788 -0.056173  0.284499   
3367  1.000000  0.035664 -0.020589  0.481677  0.203913  0.107837 -0.072252   
4503  1.000000  0.249790  0.192862  0.544004  0.187089  0.061410 -0.096554   
5971  1.000000  0.068896  0.324979  0.409920  0.239331  0.228214  0.145415   
5877  0.993403  0.810203  1.000000  0.398418 -0.314860  0.181392 -0.074023   
3225  1.000000  0.093211 -0.014894  0.444653  0.218826  0.083361 -0.081022   
2577  1.000000  0.453291  0.436329  0.604977  0.015944 -0.038978 -0.041440   
7181  1.000000 -0.442139 -0.328404  0.031452  0.056017  0.424856

# b (i)
Accuracy classification score:

Exact match: is the most strict metric, indicating the percentage of samples that have all their labels classified correctly.

In multilabel classification, this function computes subset accuracy: the set of labels predicted for a sample must exactly match the corresponding set of labels in y_true.


Compute the average Hamming loss:

Hamming loss: the fraction of the wrong labels to the total number of labels

The Hamming loss is the fraction of labels that are incorrectly predicted.

but in this question, multiclass-multioutput is not supported with sklearn


hamming score:

Another typical way to compute the accuracy is defined in (1) and (2), and less ambiguously referred to as the Hamming score (4) (since it is closely related to the Hamming loss), or label-based accuracy). It is computed as follows:

def hamming_score(y_true, y_pred, normalize=True, sample_weight=None):
    '''
    Compute the Hamming score (a.k.a. label-based accuracy) for the multi-label case
    https://stackoverflow.com/q/32239577/395857
    '''
    acc_list = []
    for i in range(y_true.shape[0]):
        set_true = set( np.where(y_true[i])[0] )
        set_pred = set( np.where(y_pred[i])[0] )
        #print('\nset_true: {0}'.format(set_true))
        #print('set_pred: {0}'.format(set_pred))
        tmp_a = None
        if len(set_true) == 0 and len(set_pred) == 0:
            tmp_a = 1
        else:
            tmp_a = len(set_true.intersection(set_pred))/\
                    float( len(set_true.union(set_pred)) )
        #print('tmp_a: {0}'.format(tmp_a))
        acc_list.append(tmp_a)
    return np.mean(acc_list)

    
    

In [93]:
from sklearn.metrics import hamming_loss
from sklearn.metrics import accuracy_score
# hamming_loss(y_true, y_pred)
# accuracy_score(y_true, y_pred)

# used in evaluating multilabel classification 
def hamming_score(y_true, y_pred):
    acc_list = []
    for i in range(y_true.shape[0]):
        set_true = set( y_true[i] )
        set_pred = set( y_pred[i] )
        #print('\nset_true: {0}'.format(set_true))
        #print('set_pred: {0}'.format(set_pred))
        tmp_a = None

        tmp_a = len(set_true.intersection(set_pred))/\
                    float( len(set_true.union(set_pred)) )
        #print('tmp_a: {0}'.format(tmp_a))
        acc_list.append(tmp_a)
    return np.mean(acc_list)


def exact_match(y_true, y_pred):
    acc_list = []
    for i in range(y_true.shape[0]):
        if (y_true[i][1] == y_pred[i][1]) and (y_true[i][0] == y_pred[i][0]) and (y_true[i][2] == y_pred[i][2]):    
            acc_list.append(1)
        else:
            acc_list.append(0)
    return np.mean(acc_list)





# i decided to use the method like this example:
import numpy as np
y_true = np.array([[1, 1], [2, 3]])
y_pred = np.array([[0, 1], [1, 2]])
hamming_loss_value = np.sum(np.not_equal(y_true, y_pred))/float(y_true.size)
accuracy_score_value = 1 - hamming_loss_value

# b (ii)

# raw attributes

# Families label

In [94]:
from sklearn import svm
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import StratifiedKFold


train_X=train.iloc[:,0:22]
train_y=train.iloc[:,22:23]
test_X=test.iloc[:,0:22]
test_y=test.iloc[:,22:23]
C_range = np.logspace(-2, 10, 13)
# the weight of the SVM penalty
gamma_range = np.logspace(-9, 3, 13)
# the width of the Gaussian Kernel
param_grid = dict(gamma=gamma_range, C=C_range)
# the process of determining
clf = svm.SVC(decision_function_shape='ovr',kernel='rbf')
cv = StratifiedKFold(n_splits=10)
grid = GridSearchCV(clf, param_grid=param_grid, cv=cv)
grid.fit(train_X, train_y)

y_pred_families = grid.predict(test_X)

y_pred = grid.predict(test_X)
print 'single classifer score'
print grid.score(test_X,test_y)
print 'single hamming loss'
print hamming_loss(test_y, y_pred)
print 'single match accuracy'
print accuracy_score(test_y, y_pred)

single classifer score
0.9921259842519685
single hamming loss
0.007874015748031496
single match accuracy
0.9921259842519685


# Genus label

In [95]:
from sklearn import svm
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import StratifiedKFold

train_X=train.iloc[:,0:22]
train_y=train.iloc[:,23:24]
test_X=test.iloc[:,0:22]
test_y=test.iloc[:,23:24]
C_range = np.logspace(-2, 10, 13)
# the weight of the SVM penalty
gamma_range = np.logspace(-9, 3, 13)
# the width of the Gaussian Kernel
param_grid = dict(gamma=gamma_range, C=C_range)
# the process of determining
clf = svm.SVC(decision_function_shape='ovr',kernel='rbf')
cv = StratifiedKFold(n_splits=10)
grid = GridSearchCV(clf, param_grid=param_grid, cv=cv)
grid.fit(train_X, train_y)

y_pred_genus = grid.predict(test_X)

y_pred = grid.predict(test_X)
print 'single classifer score'
print grid.score(test_X,test_y)
print 'single hamming loss'
print hamming_loss(test_y, y_pred)
print 'single match accuracy'
print accuracy_score(test_y, y_pred)




single classifer score
0.9879573876794813
single hamming loss
0.012042612320518759
single match accuracy
0.9879573876794813


# Species label

In [96]:
from sklearn import svm
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import StratifiedKFold

train_X=train.iloc[:,0:22]
train_y=train.iloc[:,24:25]
test_X=test.iloc[:,0:22]
test_y=test.iloc[:,24:25]
C_range = np.logspace(-2, 10, 13)
# the weight of the SVM penalty
gamma_range = np.logspace(-9, 3, 13)
# the width of the Gaussian Kernel
param_grid = dict(gamma=gamma_range, C=C_range)
# the process of determining
clf = svm.SVC(decision_function_shape='ovr',kernel='rbf')
cv = StratifiedKFold(n_splits=10)
grid = GridSearchCV(clf, param_grid=param_grid, cv=cv)
grid.fit(train_X, train_y)

y_pred_species = grid.predict(test_X)

y_pred = grid.predict(test_X)
print 'single classifer score'
print grid.score(test_X,test_y)
print 'single hamming loss'
print hamming_loss(test_y, y_pred)
print 'single match accuracy'
print accuracy_score(test_y, y_pred)


single classifer score
0.9874942102825383
single hamming loss
0.012505789717461788
single match accuracy
0.9874942102825383


# summary

In [97]:


y_true = test.iloc[:,22:25].values
y_pred_families = y_pred_families.reshape(2159,1)
y_pred_genus = y_pred_genus.reshape(2159,1)
y_pred_species =y_pred_species.reshape(2159,1)
y_pred = np.concatenate((y_pred_families,y_pred_genus,y_pred_species), axis=1)
hamming_loss_value = np.sum(np.not_equal(y_true, y_pred))/float(y_true.size)
hamming_score_value = hamming_score(y_true, y_pred)
accuracy_score_value = exact_match(y_true, y_pred)

print 'hamming loss'
print hamming_loss_value
print 'hamming score'
print hamming_score_value
print 'exact match score'
print accuracy_score_value





hamming loss
0.010807472595337347
hamming score
0.9876794812413152
exact match score
0.984251968503937


# normalized attributes

In [98]:
def norm(df):
    df_norm = (df - df.mean()) / (df.max() - df.min())
    return df_norm



# Families label

In [99]:
train_X=train.iloc[:,0:22]
train_y=train.iloc[:,22:23]
test_X=test.iloc[:,0:22]
test_y=test.iloc[:,22:23]

train_X_norm = norm(train_X)
test_X_norm = norm(test_X)

C_range = np.logspace(-2, 10, 13)
# the weight of the SVM penalty
gamma_range = np.logspace(-9, 3, 13)
# the width of the Gaussian Kernel
param_grid = dict(gamma=gamma_range, C=C_range)
# the process of determining
clf = svm.SVC(decision_function_shape='ovr',kernel='rbf')
cv = StratifiedKFold(n_splits=10)
grid = GridSearchCV(clf, param_grid=param_grid, cv=cv)
grid.fit(train_X_norm, train_y)
y_pred_families = grid.predict(test_X_norm)

y_pred = grid.predict(test_X_norm)
print 'single classifer score'
print grid.score(test_X_norm,test_y)
print 'single hamming loss'
print hamming_loss(test_y, y_pred)
print 'single match accuracy'
print accuracy_score(test_y, y_pred)



single classifer score
0.983788791106994
single hamming loss
0.01621120889300602
single match accuracy
0.983788791106994


# Genus label

In [100]:
train_X=train.iloc[:,0:22]
train_y=train.iloc[:,23:24]
test_X=test.iloc[:,0:22]
test_y=test.iloc[:,23:24]

train_X_norm = norm(train_X)
test_X_norm = norm(test_X)

C_range = np.logspace(-2, 10, 13)
# the weight of the SVM penalty
gamma_range = np.logspace(-9, 3, 13)
# the width of the Gaussian Kernel
param_grid = dict(gamma=gamma_range, C=C_range)
# the process of determining
clf = svm.SVC(decision_function_shape='ovr',kernel='rbf')
cv = StratifiedKFold(n_splits=10)
grid = GridSearchCV(clf, param_grid=param_grid, cv=cv)
grid.fit(train_X_norm, train_y)
y_pred_genus = grid.predict(test_X_norm)

y_pred = grid.predict(test_X_norm)
print 'single classifer score'
print grid.score(test_X_norm,test_y)
print 'single hamming loss'
print hamming_loss(test_y, y_pred)
print 'single match accuracy'
print accuracy_score(test_y, y_pred)

single classifer score
0.9865678554886521
single hamming loss
0.013432144511347846
single match accuracy
0.9865678554886521


# Species label

In [101]:
train_X=train.iloc[:,0:22]
train_y=train.iloc[:,24:25]
test_X=test.iloc[:,0:22]
test_y=test.iloc[:,24:25]

train_X_norm = norm(train_X)
test_X_norm = norm(test_X)

C_range = np.logspace(-2, 10, 13)
# the weight of the SVM penalty
gamma_range = np.logspace(-9, 3, 13)
# the width of the Gaussian Kernel
param_grid = dict(gamma=gamma_range, C=C_range)
# the process of determining
clf = svm.SVC(decision_function_shape='ovr',kernel='rbf')
cv = StratifiedKFold(n_splits=10)
grid = GridSearchCV(clf, param_grid=param_grid, cv=cv)
grid.fit(train_X_norm, train_y)
y_pred_species = grid.predict(test_X_norm)

y_pred = grid.predict(test_X_norm)
print 'single classifer score'
print grid.score(test_X_norm,test_y)
print 'single hamming loss'
print hamming_loss(test_y, y_pred)
print 'single match accuracy'
print accuracy_score(test_y, y_pred)

single classifer score
0.9814729041222788
single hamming loss
0.018527095877721167
single match accuracy
0.9814729041222788


# summary

In [102]:
y_true = test.iloc[:,22:25].values
y_pred_families = y_pred_families.reshape(2159,1)
y_pred_genus = y_pred_genus.reshape(2159,1)
y_pred_species =y_pred_species.reshape(2159,1)
y_pred = np.concatenate((y_pred_families,y_pred_genus,y_pred_species), axis=1)
hamming_loss_value = np.sum(np.not_equal(y_true, y_pred))/float(y_true.size)
hamming_score_value = hamming_score(y_true, y_pred)
accuracy_score_value = exact_match(y_true, y_pred)

print 'hamming loss'
print hamming_loss_value
print 'hamming score'
print hamming_score_value
print 'exact match score'
print accuracy_score_value

hamming loss
0.016056816427358345
hamming score
0.9799907364520613
exact match score
0.9698934691987031


# b(iii)

# Families label

In [103]:
train_X=train.iloc[:,0:22]
train_y=train.iloc[:,22:23]
test_X=test.iloc[:,0:22]
test_y=test.iloc[:,22:23]

train_X_norm = norm(train_X)
test_X_norm = norm(test_X)

C_range = np.logspace(-2, 10, 13)
param_grid = dict( C=C_range)
# determining the weight of the SVM penalty
clf = svm.LinearSVC(penalty='l1',dual=False,multi_class='ovr')
cv = StratifiedKFold(n_splits=10)
grid = GridSearchCV(clf, param_grid=param_grid, cv=cv)
grid.fit(train_X_norm, train_y)
y_pred_families = grid.predict(test_X_norm)

y_pred = grid.predict(test_X_norm)
print 'single classifer score'
print grid.score(test_X_norm,test_y)
print 'single hamming loss'
print hamming_loss(test_y, y_pred)
print 'single match accuracy'
print accuracy_score(test_y, y_pred)

single classifer score
0.9272811486799444
single hamming loss
0.07271885132005558
single match accuracy
0.9272811486799444


# Genus label

In [104]:
train_X=train.iloc[:,0:22]
train_y=train.iloc[:,23:24]
test_X=test.iloc[:,0:22]
test_y=test.iloc[:,23:24]

train_X_norm = norm(train_X)
test_X_norm = norm(test_X)

C_range = np.logspace(-2, 10, 13)
param_grid = dict( C=C_range)
# determining the weight of the SVM penalty
clf = svm.LinearSVC(penalty='l1',dual=False,multi_class='ovr')
cv = StratifiedKFold(n_splits=10)
grid = GridSearchCV(clf, param_grid=param_grid, cv=cv)
grid.fit(train_X_norm, train_y)
y_pred_genus = grid.predict(test_X_norm)

y_pred = grid.predict(test_X_norm)
print 'single classifer score'
print grid.score(test_X_norm,test_y)
print 'single hamming loss'
print hamming_loss(test_y, y_pred)
print 'single match accuracy'
print accuracy_score(test_y, y_pred)

single classifer score
0.9397869383974062
single hamming loss
0.06021306160259379
single match accuracy
0.9397869383974062


# Species label

In [105]:
train_X=train.iloc[:,0:22]
train_y=train.iloc[:,24:25]
test_X=test.iloc[:,0:22]
test_y=test.iloc[:,24:25]

train_X_norm = norm(train_X)
test_X_norm = norm(test_X)

C_range = np.logspace(-2, 10, 13)
param_grid = dict( C=C_range)
# determining the weight of the SVM penalty
clf = svm.LinearSVC(penalty='l1',dual=False,multi_class='ovr')
cv = StratifiedKFold(n_splits=10)
grid = GridSearchCV(clf, param_grid=param_grid, cv=cv)
grid.fit(train_X_norm, train_y)
y_pred_species = grid.predict(test_X_norm)

y_pred = grid.predict(test_X_norm)
print 'single classifer score'
print grid.score(test_X_norm,test_y)
print 'single hamming loss'
print hamming_loss(test_y, y_pred)
print 'single match accuracy'
print accuracy_score(test_y, y_pred)

single classifer score
0.9490504863362668
single hamming loss
0.05094951366373321
single match accuracy
0.9490504863362668


# summary

In [106]:
y_true = test.iloc[:,22:25].values
y_pred_families = y_pred_families.reshape(2159,1)
y_pred_genus = y_pred_genus.reshape(2159,1)
y_pred_species =y_pred_species.reshape(2159,1)
y_pred = np.concatenate((y_pred_families,y_pred_genus,y_pred_species), axis=1)
hamming_loss_value = np.sum(np.not_equal(y_true, y_pred))/float(y_true.size)
hamming_score_value = hamming_score(y_true, y_pred)
accuracy_score_value = exact_match(y_true, y_pred)

print 'hamming loss'
print hamming_loss_value
print 'hamming score'
print hamming_score_value
print 'exact match score'
print accuracy_score_value

hamming loss
0.06129380886212753
hamming score
0.9289485873089393
exact match score
0.905048633626679


# b (iv)

# Families label

In [107]:
from imblearn.over_sampling import SMOTE


sm = SMOTE()

train_X=train.iloc[:,0:22]
train_y=train.iloc[:,22:23]
test_X=test.iloc[:,0:22]
test_y=test.iloc[:,22:23]

train_X_norm = norm(train_X)
test_X_norm = norm(test_X)

train_X_norm,train_y = sm.fit_sample(train_X_norm,train_y)



C_range = np.logspace(-2, 10, 13)
param_grid = dict( C=C_range)
# determining the weight of the SVM penalty
clf = svm.LinearSVC(penalty='l1',dual=False,multi_class='ovr')
cv = StratifiedKFold(n_splits=10)
grid = GridSearchCV(clf, param_grid=param_grid, cv=cv)
grid.fit(train_X_norm, train_y)
y_pred_families = grid.predict(test_X_norm)

y_pred = grid.predict(test_X_norm)
print 'single classifer score'
print grid.score(test_X_norm,test_y)
print 'single hamming loss'
print hamming_loss(test_y, y_pred)
print 'single match accuracy'
print accuracy_score(test_y, y_pred)




single classifer score
0.8957850856878184
single hamming loss
0.10421491431218156
single match accuracy
0.8957850856878184


# Genus label

In [108]:
from imblearn.over_sampling import SMOTE


sm = SMOTE()

train_X=train.iloc[:,0:22]
train_y=train.iloc[:,23:24]
test_X=test.iloc[:,0:22]
test_y=test.iloc[:,23:24]

train_X_norm = norm(train_X)
test_X_norm = norm(test_X)

train_X_norm,train_y = sm.fit_sample(train_X_norm,train_y)



C_range = np.logspace(-2, 10, 13)
param_grid = dict( C=C_range)
# determining the weight of the SVM penalty
clf = svm.LinearSVC(penalty='l1',dual=False,multi_class='ovr')
cv = StratifiedKFold(n_splits=10)
grid = GridSearchCV(clf, param_grid=param_grid, cv=cv)
grid.fit(train_X_norm, train_y)
y_pred_genus = grid.predict(test_X_norm)

y_pred = grid.predict(test_X_norm)
print 'single classifer score'
print grid.score(test_X_norm,test_y)
print 'single hamming loss'
print hamming_loss(test_y, y_pred)
print 'single match accuracy'
print accuracy_score(test_y, y_pred)

single classifer score
0.8689207966651228
single hamming loss
0.13107920333487727
single match accuracy
0.8689207966651228


# Species label

In [109]:
from imblearn.over_sampling import SMOTE


sm = SMOTE()

train_X=train.iloc[:,0:22]
train_y=train.iloc[:,24:25]
test_X=test.iloc[:,0:22]
test_y=test.iloc[:,24:25]

train_X_norm = norm(train_X)
test_X_norm = norm(test_X)

train_X_norm,train_y = sm.fit_sample(train_X_norm,train_y)


C_range = np.logspace(-2, 10, 13)
param_grid = dict( C=C_range)
# determining the weight of the SVM penalty
clf = svm.LinearSVC(penalty='l1',dual=False,multi_class='ovr')
cv = StratifiedKFold(n_splits=10)
grid = GridSearchCV(clf, param_grid=param_grid, cv=cv)
grid.fit(train_X_norm, train_y)
y_pred_species = grid.predict(test_X_norm)

y_pred = grid.predict(test_X_norm)
print 'single classifer score'
print grid.score(test_X_norm,test_y)
print 'single hamming loss'
print hamming_loss(test_y, y_pred)
print 'single match accuracy'
print accuracy_score(test_y, y_pred)



single classifer score
0.9407132931912923
single hamming loss
0.059286706808707734
single match accuracy
0.9407132931912923


# summary

In [110]:
y_true = test.iloc[:,22:25].values
y_pred_families = y_pred_families.reshape(2159,1)
y_pred_genus = y_pred_genus.reshape(2159,1)
y_pred_species =y_pred_species.reshape(2159,1)
y_pred = np.concatenate((y_pred_families,y_pred_genus,y_pred_species), axis=1)
hamming_loss_value = np.sum(np.not_equal(y_true, y_pred))/float(y_true.size)
hamming_score_value = hamming_score(y_true, y_pred)
accuracy_score_value = exact_match(y_true, y_pred)

print 'hamming loss'
print hamming_loss_value
print 'hamming score'
print hamming_score_value
print 'exact match score'
print accuracy_score_value

hamming loss
0.09819360815192218
hamming score
0.8792959703566466
exact match score
0.824918943955535


obviously, in this question, SMOTE did not improve the result, the accuracy is lower than L1-penalized SVMs without SMOTE. The result is not so good.


# b (v) Extra Practice: 

# Families label

In [ ]:
from sklearn.multioutput import ClassifierChain
from sklearn.metrics import jaccard_similarity_score
from imblearn.over_sampling import SMOTE

sm = SMOTE()

train_X=train.iloc[:,0:22]
train_y=train.iloc[:,22:23]
test_X=test.iloc[:,0:22]
test_y=test.iloc[:,22:23]
train_y.replace({ 'Leptodactylidae' : 2, 'Dendrobatidae' : 1, 'Hylidae' : 3, 'Bufonidae' :4})
test_y.replace({ 'Leptodactylidae' : 2, 'Dendrobatidae' : 1, 'Hylidae' : 3, 'Bufonidae' :4})
train_X_norm = norm(train_X)
test_X_norm = norm(test_X)

train_X_norm,train_y = sm.fit_sample(train_X_norm,train_y)





clf = svm.LinearSVC(penalty='l1',dual=False,multi_class='ovr')
cv = StratifiedKFold(n_splits=10)
clf.fit(train_X_norm, train_y)
y_pred = clf.predict(test_X_norm)

ovr_jaccard_score = jaccard_similarity_score(test_y, y_pred)


chains = [ClassifierChain(clf,cv=cv, order='random', random_state=i)
          for i in range(10)]
for chain in chains:
    chain.fit(train_X_norm, train_y)
#  use classifier chain here

Y_pred_chains = np.array([chain.predict(test_X_norm) for chain in
                          chains])
chain_jaccard_scores = [jaccard_similarity_score(test_y, y_pred >= .5)
                        for Y_pred_chain in Y_pred_chains]

Y_pred_ensemble = Y_pred_chains.mean(axis=0)
ensemble_jaccard_score = jaccard_similarity_score(test_y,
                                                  Y_pred_ensemble >= .5)

model_scores = [ovr_jaccard_score] + chain_jaccard_scores
model_scores.append(ensemble_jaccard_score)

model_names = ('Independent',
               'Chain 1',
               'Chain 2',
               'Chain 3',
               'Chain 4',
               'Chain 5',
               'Chain 6',
               'Chain 7',
               'Chain 8',
               'Chain 9',
               'Chain 10',
               'Ensemble')

x_pos = np.arange(len(model_names))


fig, ax = plt.subplots(figsize=(7, 4))
ax.grid(True)
ax.set_title('Classifier Chain Ensemble Performance Comparison')
ax.set_xticks(x_pos)
ax.set_xticklabels(model_names, rotation='vertical')
ax.set_ylabel('Jaccard Similarity Score')
ax.set_ylim([min(model_scores) * .9, max(model_scores) * 1.1])
colors = ['r'] + ['b'] * len(chain_jaccard_scores) + ['g']
ax.bar(x_pos, model_scores, alpha=0.5, color=colors)
plt.tight_layout()
plt.show()




In [112]:
print train_y


['Leptodactylidae' 'Leptodactylidae' 'Hylidae' ... 'Hylidae' 'Hylidae'
 'Hylidae']
